# Array Detector (non-EPICS)

In [ ]:
import numpy
from pathlib import Path

Define a function that integrates directly the hardware to acquire an image and save it at a specified filepath. It should return the array shape (i.e. dimensions) of the image.

In [ ]:
def acquire_image(filepath):
    """
    This function should integrate directly with the hardware.
    
    No concepts particular to ophyd are involved here.
    Just tell the hardware to take an image, however that works.
    This function should block until acquisition is complete or
    raise if acquisition fails.
    
    It will be run on a worker thread, so it will not block
    ophyd / the RunEngine.
    """
    # For this tutorail, just generate a random image.
    from PIL import Image
    
    image = numpy.random.randint(0, 255, (512, 512)).astype('uint8')
    # Ensure the directory exists.
    Path(filepath).parent.mkdir(parents=True, exist_ok=True)
    # Save the image.
    Image.fromarray(image).save(filepath)
    return image.shape

In [ ]:
acquire_image('test.jpg')

This cell displays `test.jpg`.

![test.jpg](./test.jpg)

In [ ]:
import os
import uuid
import threading
import itertools

import requests
from ophyd import Device, Component, Signal, DeviceStatus
from ophyd.areadetector.filestore_mixins import resource_factory



class ExternalFileReference(Signal):
    """
    A pure software signal pointing to data in an external file
    
    The parent device is intended to set the value of this Signal to a datum_id.
    """
    def __init__(self, *args, shape, **kwargs):
        super().__init__(*args, **kwargs)
        self.shape = shape

    def describe(self):
        res = super().describe()
        # Tell consumers that readings from this Signal point to "external" data,
        # data that is not in-line in the reading itself.
        res[self.name].update(dict(external="FILESTORE:", dtype="array", shape=self.shape))
        return res


class Camera(Device):
    """
    An ophyd device for a camera that acquires images and saves them in files.
    """
    # We initialize the shape to [] and update it below once we know the shape
    # of the array.
    image = Component(ExternalFileReference, value="", kind="normal", shape=[])

    def __init__(self, *args, root_path, **kwargs):
        super().__init__(*args, **kwargs)
        self._root_path = root_path
        # Use this lock to ensure that we only process one "trigger" at a time.
        # Generally bluesky should care of this, so this is just an extra
        # precaution.
        self._acquiring_lock = threading.Lock()
        self._counter = None  # set to an itertools.count object when staged
        # Accumulate Resource and Datum documents in this cache.
        self._asset_docs_cache = []
        # This string is included in the Resource documents to indicate which
        # can of reader ("handler") is needed to access the relevant data.
        self._SPEC = "MY_FORMAT_SPEC"

    def stage(self):
        # Set the filepath where will be saving images.
        self._rel_path_template = f"images/{uuid.uuid4()}_%d.jpg"
        # Create a Resource document referring to this series of images that we
        # are about to take, and stash it in _asset_docs_cache.
        resource, self._datum_factory = resource_factory(
            self._SPEC, self._root_path, self._rel_path_template, {}, "posix")
        self._asset_docs_cache.append(("resource", resource))
        self._counter = itertools.count()
        return super().stage()

    def unstage(self):
        self._counter = None
        self._asset_docs_cache.clear()
        return super().unstage()

    def trigger(self):
        status = DeviceStatus(self)
        if self._counter is None:
            raise RuntimeError("Device must be staged before triggering.")
        i = next(self._counter)
        # Start a background thread to capture an image and write it to disk.
        thread = threading.Thread(target=self._capture, args=(status, i))
        thread.start()
        # Promptly return a status object, which will be marked "done" when the
        # capture completes.
        return status

    def _capture(self, status, i):
        "This runs on a background thread."
        try:
            if not self._acquiring_lock.acquire(timeout=0):
                raise RuntimeError("Cannot trigger, currently trigggering!")
            filepath = os.path.join(self._root_path, self._rel_path_template % i)
            # Kick off requests, or subprocess, or whatever with the result
            # that a file is saved at `filepath`.
            shape = acquire_image(filepath)
            self.image.shape = shape
            # Compose a Datum document referring to this specific image, and
            # stash it in _asset_docs_cache.
            datum = self._datum_factory({"index": i})
            self._asset_docs_cache.append(("datum", datum))
            self.image.set(datum["datum_id"]).wait()
            
        except Exception as exc:
            status.set_exception(exc)
        else:
            status.set_finished()
        finally:
            self._acquiring_lock.release()

    def collect_asset_docs(self):
        "Yield the documents from our cache, and reset it."
        yield from self._asset_docs_cache
        self._asset_docs_cache.clear()



class MyHandler:
    def __init__(self, resource_path):
        # resource_path is really a template string with a %d in it
        self._template = resource_path

    def __call__(self, index):
        import PIL, numpy
        filepath = str(self._template) % index
        return numpy.asarray(PIL.Image.open(filepath))

In [ ]:
camera = Camera(root_path="external_data", name="camera")
camera

## Manually walk through cycle

In [ ]:
camera.stage()

In [ ]:
status = camera.trigger()
status

In [ ]:
status

In [ ]:
camera.describe()

In [ ]:
camera.read()

In [ ]:
documents = list(camera.collect_asset_docs())

In [ ]:
camera.unstage()

## Manually inspect documents and access array data

In [ ]:
documents

In [ ]:
_, resource_document = documents[0]
_, datum_document = documents[1]
handler = MyHandler(
    Path(resource_document["root"], resource_document["resource_path"]),
    **resource_document["resource_kwargs"]
)
handler(**datum_document["datum_kwargs"])

## Use with Bluesky RunEngine and Databroker

In [ ]:
from bluesky import RunEngine
from databroker.v2 import temp

RE = RunEngine()
db = temp()
RE.subscribe(db.v1.insert)

db.register_handler("MY_FORMAT_SPEC", MyHandler)

In [ ]:
from bluesky.plans import count

In [ ]:
RE(count([camera]))

In [ ]:
run = db[-1]  # Acccess the most recent run.
dataset = run.primary.read()  # Access the dataset of its 'primary' stream.
dataset

In [ ]:
dataset["camera_image"]